In [1]:
from polygon import RESTClient
from typing import cast
from urllib3 import HTTPResponse
from config import POLYGON_API_KEY
import json
from datetime import datetime
import time
import pandas as pd

In [2]:
# client = RESTClient(POLYGON_API_KEY)

# aggs = cast(
#     HTTPResponse,
#     client.get_aggs(
#         "JPM",
#         1,
#         "day",
#         "2022-09-14",
#         "2022-09-15",
#         raw=True,
#     ),
# )

In [3]:
time.mktime(datetime(2022, 9, 15, 11, 0).timetuple())*1e9

1.663254e+18

In [4]:
client = RESTClient(POLYGON_API_KEY)

quotes_aggs = cast(
    HTTPResponse,
    client.list_quotes(
        ticker="JPM",
        timestamp_gte=datetime(2022, 9, 15, 8, 0),
        timestamp_lte=datetime(2022, 9, 16, 8, 1),
        limit=5000,
        raw=True,
    ),
)

trades_aggs = cast(
    HTTPResponse,
    client.list_trades(
        ticker="JPM",
        timestamp_gte=datetime(2022, 9, 15, 8, 0),
        timestamp_lte=datetime(2022, 9, 16, 8, 1),
        limit=5000,
        raw=True,
    ),
)

In [5]:
print(quotes_aggs.geturl())
print(quotes_aggs.status)
print(trades_aggs.geturl())
print(trades_aggs.status)
quotes_aggs_res = json.loads(quotes_aggs.data.decode("utf-8"))['results']
trades_aggs_res = json.loads(trades_aggs.data.decode("utf-8"))['results']
print(f"{len(quotes_aggs_res)} quotes")
print(f"{len(trades_aggs_res)} trades")

https://api.polygon.io/v3/quotes/JPM
200
https://api.polygon.io/v3/trades/JPM
200
5000 quotes
5000 trades


print(json.dumps(quotes_aggs_res[:2], indent=4))
print(json.dumps(quotes_aggs_res[-2:], indent=4))

print(json.dumps(trades_aggs_res[:2], indent=4))
print(json.dumps(trades_aggs_res[-2:], indent=4))

In [6]:
# https://stackoverflow.com/questions/15649942/how-to-convert-epoch-time-with-nanoseconds-to-human-readable

def nanos2dt(row, ts_col):
    dt = datetime.fromtimestamp(row[ts_col] / 1e9)
    return '{}.{:09.0f}'.format(dt.strftime('%Y-%m-%dT%H:%M:%S'), row[ts_col] % 1e9)

In [7]:
def list_col_len_gt1(row, list_col):
    return(len(row[list_col]) > 1)

In [8]:
def same_ts_dif_prices(df, ts_col, prices_col):
    df = df[[ts_col, prices_col]]
    df = df.groupby(ts_col)[prices_col].apply(set).reset_index()
    return(df[df.apply(lambda row: list_col_len_gt1(row, prices_col), axis=1)])

In [9]:
quotes_df = pd.json_normalize(quotes_aggs_res)

In [10]:
# pt_dt - participant datetime
quotes_df['pt_dt'] = quotes_df.apply(lambda row: nanos2dt(row, 'participant_timestamp'), axis=1)
quotes_df['st_dt'] = quotes_df.apply(lambda row: nanos2dt(row, 'sip_timestamp'), axis=1)
# quotes_df

In [12]:
same_ts_dif_prices(quotes_df, 'pt_dt', 'ask_price')

,pt_dt,ask_price
924,2022-09-15T15:59:26.164717056,"{117.91, 117.92}"
2060,2022-09-15T15:59:45.335085568,"{117.9, 117.91}"
2371,2022-09-15T15:59:54.647067392,"{117.86, 117.87}"


In [17]:
same_ts_dif_prices(quotes_df, 'pt_dt', 'bid_price')

,pt_dt,bid_price
838,2022-09-15T15:59:24.277662976,"{117.91, 117.9}"
993,2022-09-15T15:59:28.891982848,"{117.92, 117.91}"
1180,2022-09-15T15:59:31.894486016,"{117.88, 117.89}"
1537,2022-09-15T15:59:38.133655296,"{117.92, 117.91}"
1616,2022-09-15T15:59:39.847317760,"{117.93, 117.92}"
1622,2022-09-15T15:59:39.848062976,"{117.92, 117.91}"
1775,2022-09-15T15:59:40.730137344,"{117.92, 117.91}"
1792,2022-09-15T15:59:40.730839296,"{117.91, 117.9}"
2348,2022-09-15T15:59:53.815138560,"{117.87, 117.86}"
2574,2022-09-15T16:45:15.372142080,"{117.65, 117.26}"


In [13]:
trades_df = pd.json_normalize(trades_aggs_res)

In [14]:
# pt_dt - participant datetime
trades_df['pt_dt'] = trades_df.apply(lambda row: nanos2dt(row, 'participant_timestamp'), axis=1)
trades_df['st_dt'] = trades_df.apply(lambda row: nanos2dt(row, 'sip_timestamp'), axis=1)
# quotes_df

In [16]:
same_ts_dif_prices(trades_df, 'pt_dt', 'price')

,pt_dt,price
85,2022-09-15T15:56:12.870061824,"{118.105, 118.1}"
109,2022-09-15T15:56:14.827749888,"{118.085, 118.08}"
114,2022-09-15T15:56:14.828572672,"{118.085, 118.08}"
123,2022-09-15T15:56:14.830793984,"{118.085, 118.08}"
218,2022-09-15T15:56:19.901172992,"{117.995, 117.99}"
...,...,...
4399,2022-09-16T07:33:39.322437120,"{116.98, 116.96, 116.95, 116.99, 117.0, 116.94..."
4403,2022-09-16T07:33:39.322980096,"{117.01, 117.02}"
4404,2022-09-16T07:33:46.407084032,"{117.01, 117.0}"
4415,2022-09-16T07:45:29.941544448,"{117.05, 117.02}"


In [20]:
same_ts_dif_prices(quotes_df, 'st_dt', 'ask_price')

,st_dt,ask_price
146,2022-09-15T15:59:11.863169024,"{117.92, 117.93}"
698,2022-09-15T15:59:19.679367168,"{117.92, 117.93}"
836,2022-09-15T15:59:24.280034560,"{117.92, 117.91}"
905,2022-09-15T15:59:26.164956672,"{117.91, 117.92}"
1790,2022-09-15T15:59:40.732651008,"{117.9, 117.91}"
2335,2022-09-15T15:59:54.647411968,"{117.86, 117.87}"
2485,2022-09-15T15:59:57.804101632,"{117.86, 117.88}"
4929,2022-09-16T08:00:56.385857280,"{117.14, 117.1}"


In [21]:
same_ts_dif_prices(quotes_df, 'st_dt', 'bid_price')

,st_dt,bid_price
289,2022-09-15T15:59:14.380062720,"{117.9, 117.89}"
823,2022-09-15T15:59:24.278009600,"{117.91, 117.9}"
974,2022-09-15T15:59:28.892329216,"{117.92, 117.91}"
1513,2022-09-15T15:59:38.133999360,"{117.92, 117.91}"
1593,2022-09-15T15:59:39.847663360,"{117.93, 117.92}"
1598,2022-09-15T15:59:39.848410112,"{117.92, 117.91}"
1747,2022-09-15T15:59:40.730486016,"{117.92, 117.91}"
1780,2022-09-15T15:59:40.731869184,"{117.9, 117.89}"


In [22]:
same_ts_dif_prices(trades_df, 'st_dt', 'price')

,st_dt,price
116,2022-09-15T15:56:14.827804928,"{118.085, 118.08}"
124,2022-09-15T15:56:14.828942592,"{118.085, 118.08}"
184,2022-09-15T15:56:16.531939840,"{118.02, 118.03}"
522,2022-09-15T15:56:50.264137216,"{117.97, 117.96}"
619,2022-09-15T15:56:59.180049920,"{118.005, 118.0}"
681,2022-09-15T15:57:07.361438976,"{117.975, 117.97}"
683,2022-09-15T15:57:07.361454080,"{117.97, 117.975}"
726,2022-09-15T15:57:10.187369984,"{117.975, 117.98}"
785,2022-09-15T15:57:14.129388800,"{118.0, 118.01}"
808,2022-09-15T15:57:16.635493120,"{117.995, 117.99}"
